In [4]:
import pandas as pd
import numpy as np
import re
from imblearn.metrics import classification_report_imbalanced

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV


from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

In [147]:
df_dummy=pd.read_csv('training.1600000.processed.noemoticon.csv',sep=',',header = None,skiprows=100,encoding='latin-1')

In [109]:
df.loc[:][0].value_counts()

4    800000
0    799900
Name: 0, dtype: int64

In [110]:
df.drop(df.columns[1:5],axis=1,inplace=True)

In [132]:
df.loc[0][5]

['body',
 'of',
 'miss',
 'northern',
 'calif',
 'girl',
 'found',
 'police',
 'have',
 'found',
 'the',
 'remains',
 'of',
 'a',
 'miss',
 'northern',
 'california',
 'girl']

In [120]:
df.loc[3][5]

'watching  '

In [113]:
re.sub(r"(?:\@|https?\://)\S+", "", a)

' Body Of Missing Northern Calif. Girl Found: Police have found the remains of a missing Northern California girl .. '

In [114]:
for i in range(len(df.loc[:][5])):
    df.at[i,5]=re.sub(r"(?:\@|https?\://)\S+", "", df.loc[i][5])

In [119]:
for i in range(len(df.loc[:][5])):
    df.at[i,5]=re.sub(r"&[^\s]*;","", df.loc[i][5])

In [116]:
re.sub(r"&[^\s]*;","",df.loc[3][5])

'watching  '

In [122]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
tokenizer.tokenize(df.loc[0][5])

['Body',
 'Of',
 'Missing',
 'Northern',
 'Calif',
 'Girl',
 'Found',
 'Police',
 'have',
 'found',
 'the',
 'remains',
 'of',
 'a',
 'missing',
 'Northern',
 'California',
 'girl']

In [123]:
for i in range(len(df.loc[:][5])):
    #final_data.title[i]=tokenizer.tokenize(final_data.title[i])
    df.at[i,5]=tokenizer.tokenize(df.at[i,5])

In [124]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import string
from nltk import pos_tag,word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

In [130]:
dummy_df=df
dummy_df.loc[88][5][5]='poop'
dummy_df.loc[88][5][5]

'poop'

In [133]:
for i in range(len(df.loc[:][5])):
    #for j in range(len(final_data.title[i])):
        #final_data.title[i][j]=lmtzr.lemmatize(final_data.title[i][j].lower())
    for j in range(len(df.loc[i][5][:])):
        for word, tag in pos_tag(word_tokenize(df.loc[i][5][j].lower())):
             wntag = tag[0].lower()
             wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
             if not wntag:
                 lemma = word
             else:
                 lemma = lmtzr.lemmatize(word, wntag)
        #print(dummy_df.description[i][j])
        df.loc[i][5][j]=lemma

In [13]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
vectorizer1 = CountVectorizer(stop_words='english')
vectorizer2 = TfidfVectorizer(stop_words='english')
#for i in range(len(df.loc[:][5])):
    #df.at[i,5] = " ".join(df.loc[i][5])   

In [15]:
df.loc[:][5]

0          body of miss northern calif girl found police ...
1          i hope they will increase the capacity fast ye...
2                                behind on my class for work
3                                                      watch
4          remember my bum leg strike back this time its ...
                                 ...                        
1599895    just woke up have no school be the best feel ever
1599896      thewdb com very cool to hear old walt interview
1599897    be you ready for your mojo makeover ask me for...
1599898    happy th birthday to my boo of alll time tupac...
1599899                                 happy charitytuesday
Name: 5, Length: 1599900, dtype: object

In [162]:
df.loc[0][5]=" ".join(df.loc[0][5])

c:\users\shubh\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
x_train_count = count_vectorizer.fit_transform(x_train)
x_train_count.shape
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
x_train_tfidf = tfidf.fit_transform(x_train_count)
x_train_tfidf.shape

#fv=vectorizer2.fit_transform(df.loc[:][5])

(1071933, 193351)

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.loc[:][5], df.loc[:][0],
                                                    test_size=0.33,
                                                    random_state=0,
                                                    stratify=df.loc[:][0])

In [18]:
from sklearn.pipeline import  Pipeline
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(x_train_tfidf, y_train)
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

text_clf = text_clf.fit(x_train, y_train)

In [20]:
predictions = text_clf.predict(x_test)
predictions

array([0, 4, 0, ..., 0, 0, 4], dtype=int64)

In [21]:
from sklearn.metrics import f1_score, accuracy_score
accuracy = accuracy_score(y_test,predictions)
print(accuracy)

f1 = f1_score(y_test,predictions,average='weighted')
print(f1)

0.7693416444588392
0.7692783388621155


In [22]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('cvec', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42))])
text_clf_svm = text_clf_svm.fit(x_train, y_train)
predictions_svm = text_clf_svm.predict(x_test)

f1_score_svm = f1_score(y_test, predictions_svm, average = 'weighted')
print(f'SVM score it is: {f1_score_svm}')

SVM score it is: 0.7443217989749341


In [27]:
model_mult_nb = MultinomialNB()
pipe1 = Pipeline([('cv',count_vectorizer),
                ('model',model_mult_nb)
])
params = {'cv__stop_words': [['and','is',]]}
gs = GridSearchCV(pipe1,param_grid=params, cv=5)
print(gs.fit(x_train,y_train))
print(gs.best_params_)
print("Train Score: ", round(gs.best_score_,4))
print("Train Score: ", round(gs.score(x_test,y_test),4))

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cv', CountVectorizer()),
                                       ('model', MultinomialNB())]),
             param_grid={'cv__stop_words': [['and', 'is']]})
{'cv__stop_words': ['and', 'is']}
Train Score:  0.7763
Train Score:  0.7773


In [8]:
X_train_vec = count_vectorizer.fit_transform(x_train)
X_test_vec = count_vectorizer.transform(x_test)
rf = RandomForestClassifier(n_estimators=100,random_state=42)
rf.fit(X_train_vec,y_train)
print("Cross Val Score: ",cross_val_score(rf,X_train_vec,y_train,cv=5).mean())
print("Train Score: ", round(rf.score(X_train_vec,y_train),4))
print("Train Score: ", round(rf.score(X_test_vec,y_test),4))

KeyboardInterrupt: 

In [ ]:
y_pred=rf.predict(X_test_vec)
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

f1 = f1_score(y_test,y_pred,average='weighted')
print(f1)
print(classification_report_imbalanced(y_test, y_pred))

In [175]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(fit_prior=False)

In [176]:
model.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False)

In [177]:
model.score(x_test,y_test)

0.7544827612331831

In [182]:
def parse_str(user_post):
    user_post=re.sub(r"(?:\@|https?\://)\S+", "", user_post)
    user_post=re.sub(r"&[^\s]*;","",user_post)
    user_post=tokenizer.tokenize(user_post)
    for i in range(len(user_post)):
        for word, tag in pos_tag(word_tokenize(user_post[i].lower())):
             wntag = tag[0].lower()
             wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
             if not wntag:
                 lemma = word
             else:
                 lemma = lmtzr.lemmatize(word, wntag)
        #print(dummy_df.description[i][j])
        user_post[i]=lemma
    return " ".join(user_post)

In [188]:
parse_str('I am very angry at my teacher. he kicked me out of class.')

'i be very angry at my teacher he kick me out of class'

In [186]:
df2.tail()

,0,5
1599895,4,just woke up have no school be the best feel ever
1599896,4,thewdb com very cool to hear old walt interview
1599897,4,be you ready for your mojo makeover ask me for...
1599898,4,happy th birthday to my boo of alll time tupac...
1599899,4,happy charitytuesday


In [206]:
test_data=df2.append({5:parse_str('i was very surprised when i saw her very happy in her new attire.')},ignore_index=True)

In [207]:
test_data.loc[:][5]

0          body of miss northern calif girl found police ...
1          i hope they will increase the capacity fast ye...
2                                behind on my class for work
3                                                      watch
4          remember my bum leg strike back this time its ...
                                 ...                        
1599896      thewdb com very cool to hear old walt interview
1599897    be you ready for your mojo makeover ask me for...
1599898    happy th birthday to my boo of alll time tupac...
1599899                                 happy charitytuesday
1599900    i be very surprised when i saw her very happy ...
Name: 5, Length: 1599901, dtype: object

In [208]:
fv2=vectorizer2.transform(test_data.loc[:][5])

In [209]:
a,b=fv2.shape
model.predict(fv2[a-1])

array([4], dtype=int64)

In [1]:
import pickle

In [13]:
infile=open('data.pkl','rb')
#new_mod = pickle.load(infile)

In [14]:
model_NB=pickle.load(infile)

In [21]:
outp=model_NB.predict(x_test)

In [33]:
type(x_test)

pandas.core.series.Series

In [65]:
pred=model_NB.predict(x_test)

In [55]:
type(x_test.iloc[[2321]])

pandas.core.series.Series

In [40]:
pd.set_option('display.max_colwidth', -1)

In [58]:
a=pd.Series()

In [60]:
a=a.append(pd.Series('good day'),ignore_index=True)

In [62]:
a=a.append(pd.Series('bad day'),ignore_index=True)

In [67]:
from collections import Counter
b=Counter(pred)

In [70]:
b[0]/(b[0]+b[4])

0.5176668238734617

In [69]:
b

Counter({0: 273311, 4: 254656})